In [2]:
import numpy as np
from scipy.stats import uniform

def gen_bernulli(n,p):
    """    
    Generates bernulli sequence
    """
    i = 1
    while i <= n:
        if (uniform.rvs() < p):
            yield 1
        else:
            yield 0
        i += 1
#end of gen_bernulli

def binomial(n,p):
    """
    Generates value of random varible ~ Bin(n,p)
    """
    init_list = list(gen_bernulli(n,p))
    return np.sum(np.array(init_list))
#end of binomial

print(binomial(10,0.5))

#try to plot histogram with seaborn
#write latex 

4
